In [ ]:
import sys
import os
from lerobot.policies.pi0.modeling_pi0 import PI0Policy

def auto_model_fn(path, **kwargs):  # noqa
    sys.path.append(path)
    map_location = kwargs.pop("map_location", "cpu")
    return PI0Policy.from_pretrained(path, **kwargs).to(map_location)

pi0_path = os.path.abspath("../checkpoints/pi0_base")
policy=auto_model_fn(pi0_path,
                     map_location="cuda",
                     local_files_only=True,)


In [ ]:
import pandas as pd
data_path = '../datasets/ur5/outputs/data/chunk_000/episode_000001.parquet'
ds=pd.read_parquet(data_path)
records = ds.to_dict(orient='records')
data = records[0]

In [ ]:
import imageio
import json
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
frame_index = 1
video_path = '../datasets/ur5/outputs/videos/chunk_000/observation.images.image/episode_000001.mp4'
reader = imageio.get_reader(video_path)

frame1 = reader.get_data(frame_index)
image_tensor = torch.from_numpy(frame1).permute(2, 0, 1).unsqueeze(0).to(device).float()

In [ ]:
observation = {
    "observation.state": torch.tensor(data['observation.state']).unsqueeze(0).to(device).float(),
    'observation.images.image': image_tensor,
    "task": ['pick up the red cube'], 
}

In [11]:

from huggingface_hub import login

from huggingface_hub import snapshot_download

# 设置你的数据集 repo ID
repo_id = 'amylingchen/Robotic_Vision_Laboratory_ur5_v21'

# 设置本地缓存路径
local_dir ="../datasets/amylingchen/Robotic_Vision_Laboratory_ur5_v21"    
snapshot_download(
    repo_id=repo_id,
    repo_type="dataset",
    local_dir=local_dir,
    local_dir_use_symlinks=False,  # 复制而不是软链，适合在远程服务器使用
    resume_download=True,
    # ignore_patterns=["*.mp4"]  # 如果不想下载视频可加上，或者删掉这个行全部下载
)

Fetching 1 files: 100%|██████████| 1/1 [00:00<00:00,  7.58it/s]


'/home/lxc4866/project/pi0/datasets/amylingchen/Robotic_Vision_Laboratory_ur5_v21'

In [16]:
from huggingface_hub import upload_folder

# 设置你的数据集 repo ID
repo_id = 'amylingchen/Robotic_Vision_Laboratory_ur5_v30'

# 设置本地缓存路径
local_dir ="../datasets/amylingchen/Robotic_Vision_Laboratory_ur5_v30"  

upload_folder(
    folder_path=local_dir,
    repo_id=repo_id,
    repo_type="dataset",
    # token='',  # 必须是 write 权限的 token

)

Processing Files (4 / 4): 100%|██████████| 34.0MB / 34.0MB, 22.8MB/s  
New Data Upload: 100%|██████████| 16.3MB / 16.3MB, 13.6MB/s  


CommitInfo(commit_url='https://huggingface.co/datasets/amylingchen/Robotic_Vision_Laboratory_ur5_v30/commit/aeafbe78742ee7836e738652e3870352a52c660b', commit_message='Upload folder using huggingface_hub', commit_description='', oid='aeafbe78742ee7836e738652e3870352a52c660b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/amylingchen/Robotic_Vision_Laboratory_ur5_v30', endpoint='https://huggingface.co', repo_type='dataset', repo_id='amylingchen/Robotic_Vision_Laboratory_ur5_v30'), pr_revision=None, pr_num=None)

In [15]:
from huggingface_hub import HfApi
api = HfApi()

api.create_tag(
    repo_id="amylingchen/Robotic_Vision_Laboratory_ur5_v21",
    tag="v2.1",
    repo_type="dataset"
)

In [3]:
## 上传模型到 Hugging Face Hub


from huggingface_hub import Repository

repo = Repository(
    local_dir='../outputs/pi0_ur5',
    clone_from="amylingchen/pi0_ur5_80k",
    git_user="amylingchen",
    git_email="linglingchen9611@gmail.com"
)



/home/lxc4866/anaconda3/envs/lerobot/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/amylingchen/pi0_ur5_80k into local empty directory.


In [ ]:
! cp -r ../outputs/pi0_training_1/checkpoints/080000/pretrained_model/* ../outputs/pi0_ur5

In [4]:
repo.push_to_hub(commit_message="Upload fine-tuned model")

Upload file model.safetensors: 6.54GB [02:38, 46.3MB/s]                            To https://huggingface.co/amylingchen/pi0_ur5_80k
   100a76f..8e11f49  main -> main

Upload file model.safetensors: 100%|██████████| 6.53G/6.53G [02:39<00:00, 44.0MB/s]
Upload file policy_postprocessor_step_0_unnormalizer_processor.safetensors: 100%|██████████| 3.23k/3.23k [02:39<00:00, 20.8B/s]


'https://huggingface.co/amylingchen/pi0_ur5_80k/commit/8e11f4973ca72984ec106b4210bb8516b2f144bf'